In [1]:
!git clone https://github.com/AI4Finance-LLC/FinRL-Library.git

fatal: destination path 'FinRL-Library' already exists and is not an empty directory.


In [2]:
%cd FinRL-Library/
!pip install -r requirements.txt
!pip install nest-asyncio

/home/notebooks/FinRL-Library


In [3]:
!pip install -U ipython
# !pip install colorama

In [4]:
%cd FinRL-Library/

[Errno 2] No such file or directory: 'FinRL-Library/'
/home/notebooks/FinRL-Library


In [5]:
import nest_asyncio
nest_asyncio.apply()

In [6]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
# matplotlib.use('Agg')
import datetime

%matplotlib inline
from finrl.config import config
from finrl.marketdata.yahoodownloader import YahooDownloader
from finrl.preprocessing.preprocessors import FeatureEngineer
from finrl.preprocessing.data import data_split
from finrl.env.env_stocktrading import StockTradingEnv
from finrl.model.models import DRLAgent
from finrl.trade.backtest import backtest_stats, backtest_plot, get_daily_return, get_baseline

from pprint import pprint

import sys
sys.path.append("../FinRL-Library")

import itertools

/opt/conda/lib/python3.6/site-packages/pyfolio/pos.py:27: UserWarning: Module "zipline.assets" not found; multipliers will not be applied to position notionals.
  'Module "zipline.assets" not found; multipliers will not be applied'


In [7]:
import os
if not os.path.exists("./" + config.DATA_SAVE_DIR):
    os.makedirs("./" + config.DATA_SAVE_DIR)
if not os.path.exists("./" + config.TRAINED_MODEL_DIR):
    os.makedirs("./" + config.TRAINED_MODEL_DIR)
if not os.path.exists("./" + config.TENSORBOARD_LOG_DIR):
    os.makedirs("./" + config.TENSORBOARD_LOG_DIR)
if not os.path.exists("./" + config.RESULTS_DIR):
    os.makedirs("./" + config.RESULTS_DIR)

In [8]:
from finrl.config.configuration import Configuration
from finrl.config.directory_operations import create_userdata_dir
from finrl.commands import start_download_cryptodata, start_download_stockdata

In [9]:
#### CREATE USER DATA DIRECTORY IN DESIGNATED PATH, IF NO NAME INDICATED DEFAULT TO user_data
####### create dir to false if only to check existence of directory
create_userdata_dir("./user_data",create_dir=True)


# ###### Pull Configuration File (using finrl/config/configuration.py)
config = Configuration.from_files(["../config.json"])

In [10]:
from finrl.commands import start_download_cryptodata, start_download_stockdata, start_list_markets

#ARGS_LIST_PAIRS = ["exchange", "print_list", "list_pairs_print_json", "print_one_column",
#                   "print_csv", "base_currencies", "quote_currencies", "list_pairs_all"]

ARGS_LIST_PAIRS = {"exchange":config["exchange"]["name"], "quote_currencies":"BNB"}

x = start_list_markets(ARGS_LIST_PAIRS)


Exchange Binance has 111 active markets with B, N, B as quote currencies:
+-----------+------------+--------+---------+----------+-----------+
|        Id |     Symbol |   Base |   Quote |   Active |   Is pair |
|-----------+------------+--------+---------+----------+-----------|
|   AAVEBNB |   AAVE/BNB |   AAVE |     BNB |     True |      True |
|    ADABNB |    ADA/BNB |    ADA |     BNB |     True |      True |
|   ALGOBNB |   ALGO/BNB |   ALGO |     BNB |     True |      True |
|  ALPHABNB |  ALPHA/BNB |  ALPHA |     BNB |     True |      True |
|   ANKRBNB |   ANKR/BNB |   ANKR |     BNB |     True |      True |
|    ANTBNB |    ANT/BNB |    ANT |     BNB |     True |      True |
|   ARPABNB |   ARPA/BNB |   ARPA |     BNB |     True |      True |
|   ATOMBNB |   ATOM/BNB |   ATOM |     BNB |     True |      True |
|    AVABNB |    AVA/BNB |    AVA |     BNB |     True |      True |
|   AVAXBNB |   AVAX/BNB |   AVAX |     BNB |     True |      True |
|    AXSBNB |    AXS/BNB |  

In [11]:
from finrl.tools.coin_search import *
import json
#Search top Selling Coins based on Volume
coins = coinSearch("BNB", top=20)
print(coins)
#Add them to config file Pair_whitelist
coins_to_json("../config.json", coins)

# reintialize config
config = Configuration.from_files(["../config.json"])

#make sure the pairs are equal...
print(config["exchange"]["pair_whitelist"])

['HOT/BNB', 'WIN/BNB', 'BTT/BNB', 'MBL/BNB', 'VET/BNB', 'RSR/BNB', 'RVN/BNB', 'JST/BNB', 'DGB/BNB', 'VTHO/BNB', 'COS/BNB', 'BLZ/BNB', 'ONE/BNB', 'CHZ/BNB', 'FTM/BNB', 'COTI/BNB', 'CHR/BNB', 'TRX/BNB', 'BNBDOWN/USDT', 'XLM/BNB']
['HOT/BNB', 'WIN/BNB', 'BTT/BNB', 'MBL/BNB', 'VET/BNB', 'RSR/BNB', 'RVN/BNB', 'JST/BNB', 'DGB/BNB', 'VTHO/BNB', 'COS/BNB', 'BLZ/BNB', 'ONE/BNB', 'CHZ/BNB', 'FTM/BNB', 'COTI/BNB', 'CHR/BNB', 'TRX/BNB', 'BNBDOWN/USDT', 'XLM/BNB']


In [12]:
ARGS_DOWNLOAD_DATA = {'config': ['../config.json'], 'datadir': None, 
                      'user_data_dir': None, 'pairs': None, 'pairs_file': None, 
                      'days': 365, 'timerange': None, 
                      'download_trades': False, 'exchange': 'binance', 
                      'timeframes': ['4h'], 'erase': False, 
                      'dataformat_ohlcv': None, 'dataformat_trades': None}

# ######## downloads data to our local data repository as dictated by our config, or we could overide it using 'datadir'
start_download_cryptodata(ARGS_DOWNLOAD_DATA)

In [13]:
from finrl.data.fetchdata import FetchData
import pandas as pd
from finrl.config import TimeRange

In [14]:
df = FetchData(config).fetch_data_crypto()
print(df.head)

coin HOT_BNB not available
coin WIN_BNB not available
coin BTT_BNB not available
coin MBL_BNB not available
coin VET_BNB not available
coin RSR_BNB not available
coin RVN_BNB not available
coin JST_BNB not available
coin DGB_BNB not available
coin VTHO_BNB not available
coin COS_BNB not available
coin BLZ_BNB not available
coin ONE_BNB not available
coin CHZ_BNB not available
coin FTM_BNB not available
coin COTI_BNB not available
coin CHR_BNB not available
coin TRX_BNB not available
coin BNBDOWN_USDT not available
coin XLM_BNB not available
(0, 0)
<bound method NDFrame.head of Empty DataFrame
Columns: []
Index: []>


In [15]:
df.head()

""


In [16]:
df.sort_values(['date','tic'],ignore_index=True).head()

fe = FeatureEngineer(
                    use_technical_indicator=True,
                    tech_indicator_list = config["TECHNICAL_INDICATORS_LIST"],
                    use_turbulence=False,
                    user_defined_feature = False)

processed = fe.preprocess_data(df)
processed.columns

KeyError: 'date'

In [ ]:
print(processed.head())
processed.shape

In [ ]:
print(processed.tail())
processed.shape

In [ ]:
list_ticker = processed["tic"].unique().tolist()
list_date = list(pd.date_range(processed['date'].min(),processed['date'].max(), freq="4h").astype(str))
combination = list(itertools.product(list_date,list_ticker))
list_date

In [ ]:
processed["date"] = processed["date"].astype(str)
processed_full = pd.DataFrame(combination,columns=["date","tic"]).merge(processed,on=["date","tic"],how="left")
processed_full = processed_full[processed_full['date'].isin(processed['date'])]
processed_full = processed_full.fillna(0)
processed_full.sort_values(['date','tic'],ignore_index=True).head(10)

In [ ]:
processed_full.head(5)

In [ ]:
processed_full.tail()

In [ ]:
#### DATA SPLIT TRAIN TRADE RATIO 70/30

trade_len = int(len(processed_full)*0.7)
train_len = int(len(processed_full)*0.3+1)
total = trade_len+train_len
print(f'trade length:{trade_len}, train length: {train}, for total len of {total} of {len(processed_full)}')

In [ ]:
processed_full.date.factorize()[0]

In [ ]:
train = data_split(processed_full, processed_full.date.min(),processed_full.date.loc[trade_len])
trade = data_split(processed_full, processed_full.date.loc[trade_len],processed_full.date.max())
print(len(train))
print(len(trade))

In [ ]:
# processed_imputed = processed_full[processed_full.columns[processed_full.isna().any().tolist()]]

# processed_imputed = processed_imputed.replace([np.inf, -np.inf], np.nan)

In [ ]:
# from sklearn.impute import KNNImputer

# imputer = KNNImputer(n_neighbors=1)
# imputed = imputer.fit_transform(processed_imputed)

# df_knn = pd.DataFrame(imputed, columns=processed_imputed.columns.tolist())

# for i in processed_imputed.columns.tolist():
#     processed_imputed[i] = df_knn[i]

In [ ]:
# print(processed_imputed.isna().any())

In [ ]:
# processed_imputed.head()

In [ ]:
# processed_imputed.tail()

In [ ]:
# from sklearn.model_selection import train_test_split 
# X = processed_imputed.astype(float).to_numpy()
# train, trade = train_test_split(X, test_size=0.33, shuffle=False)
# train = pd.DataFrame(train)
# trade = pd.DataFrame(trade)
# print(train.shape)
# print(trade.shape)
# # train = data_split(processed_full, '2018-05-16','2020-05-16')
# # trade = data_split(processed_full, '2020-05-17','2021-02-10')

In [ ]:
# processed_full.date.head()

In [ ]:
# train[["date","tic"]] = processed_full[["date","tic"]].iloc[:len(train),:]
# train.columns = processed_full.columns
# train = train.sort_values(["date", "tic"], ignore_index=True)
# train.index = train.date.factorize()[0]
# trade[["date","tic"]] = processed_full[["date","tic"]].iloc[:len(trade),:]
# trade.columns = processed_full.columns
# trade = trade.sort_values(["date", "tic"], ignore_index=True)
# trade.index = trade.date.factorize()[0]

In [ ]:
train.head()

In [ ]:
trade.head()

In [ ]:
train.info()

In [ ]:
len(train.tic.unique())

In [ ]:
config["TECHNICAL_INDICATORS_LIST"]

In [ ]:
stock_dimension = len(train.tic.unique())
state_space = 1 + 2*stock_dimension + len(config["TECHNICAL_INDICATORS_LIST"])*stock_dimension
print(f"Stock Dimension: {stock_dimension}, State Space: {state_space}")

env_kwargs = {
    "hmax": 100, 
    "initial_amount": 1000000, 
    "buy_cost_pct": 0.001,
    "sell_cost_pct": 0.001,
    "state_space": state_space, 
    "stock_dim": stock_dimension, 
    "tech_indicator_list": config["TECHNICAL_INDICATORS_LIST"], 
    "action_space": stock_dimension, 
    "reward_scaling": 1e-4
    
}
e_train_gym = StockTradingEnv(df = train, **env_kwargs)

In [ ]:
env_train, _ = e_train_gym.get_sb_env()
print(type(env_train))

In [ ]:
agent = DRLAgent(env = env_train)
PPO_PARAMS = {
    "n_steps": 2048,
    "ent_coef": 0.01,
    "learning_rate": 0.00025,
    "batch_size": 128,
}
model_ppo = agent.get_model("ppo",model_kwargs = PPO_PARAMS)


In [ ]:
trained_ppo = agent.train_model(model=model_ppo, 
                             tb_log_name='ppo',
                             total_timesteps=100000)

In [ ]:
e_trade_gym = StockTradingEnv(df = trade, **env_kwargs)
# env_trade, obs_trade = e_trade_gym.get_sb_env()

df_account_value, df_actions = DRLAgent.DRL_prediction(
    model=model_ppo, 
    environment = e_trade_gym)

In [ ]:
df_account_value.shape

In [ ]:
df_account_value.head()

In [ ]:
df_actions.head()

In [ ]:
print("==============Get Backtest Results===========")
now = datetime.datetime.now().strftime('%Y%m%d-%Hh%M')

perf_stats_all = backtest_stats(account_value=df_account_value)
perf_stats_all = pd.DataFrame(perf_stats_all)
# perf_stats_all.to_csv("./"+config.RESULTS_DIR+"/perf_stats_all_"+now+'.csv')